In [ ]:
题目目录（覆盖常见图论模型）
题号	类型	题目名称
1	最短路径（单源）	城市最短送货路径
2	最短路径（多源）	所有城市之间最短通路
3	最小生成树	电网建设成本最小化
4	拓扑排序（DAG）	项目任务安排顺序
5	连通分量	网络中断检测
6	网络最大流	河道最大运水量
7	二分图匹配	工作岗位分配
8	图的着色（染色问题）	会议安排冲突检测
9	Euler 回路 / Hamilton 回路	回收路线设计
10	强连通分量	网站相互可达性分析

In [ ]:
1. 最短路径（单源）
🧾 题目：城市送货路径优化
某物流公司要从城市 A 向其他城市发货，道路为双向通行，每条路有运输时间，求 A 到所有城市的最短路径。
A - B (2)
A - C (5)
B - C (1)
B - D (3)
C - D (2)
D - E (1)


In [14]:
import networkx as nx#调用图论库
G = nx.Graph()#建立无向图
l = [
    ('A', 'B', 2),#起点，终点，权重
    ('A', 'C', 5),
    ('B', 'C', 1),
    ('B', 'D', 3),
    ('C', 'D', 2),
    ('D', 'E', 1),
]#构建边

G.add_weighted_edges_from(l)#构建图

# 求最短路径
path = nx.shortest_path(G,'A',weight='weight')#路径(传入参数为：图，起点，终点，权重)，如果不指定weight默认为1
length = nx.shortest_path_length(G,'A',weight='weight')#路径长度

path,length


({'A': ['A'],
  'B': ['A', 'B'],
  'C': ['A', 'B', 'C'],
  'D': ['A', 'B', 'D'],
  'E': ['A', 'B', 'D', 'E']},
 {'A': 0, 'B': 2, 'C': 3, 'D': 5, 'E': 6})

In [ ]:
✅ 2. 最短路径（多源）
题目名称：所有城市之间最短通路
背景： 政府要评估城市间通行效率，边表示路程，求任意两个城市间最短距离

A - B (3)
A - C (1)
B - C (7)
B - D (5)
C - D (2)
D - E (1)


In [24]:
G=nx.Graph()
l=[('A','B',3),('A','C',1),('B','C',7),('B','D',5),('C','D',2),('D','E',1)]
G.add_weighted_edges_from(l)
d=nx.floyd_warshall(G)#生成floyd的最小距离矩阵
for u in d:
    for v in d[u]:
        print(f"{u} → {v}: {d[u][v]}")
#依次遍历两个节点间的最短距离

A → A: 0
A → B: 3
A → C: 1
A → D: 3
A → E: 4
B → B: 0
B → A: 3
B → C: 4
B → D: 5
B → E: 6
C → C: 0
C → A: 1
C → B: 4
C → D: 2
C → E: 3
D → D: 0
D → B: 5
D → C: 2
D → E: 1
D → A: 3
E → E: 0
E → D: 1
E → A: 4
E → B: 6
E → C: 3


In [ ]:
✅ 3. 最小生成树
题目名称：电网建设成本最小化
背景： 城市间建电缆，边表示成本，求连接所有城市的最小花费。

A - B (4)
A - C (1)
B - C (2)
B - D (5)
C - D (8)
D - E (3)


In [31]:
G=nx.Graph()
l=[('A','B',4),('A','C',1),('B','C',2),('B','D',5),('C','D',8),('D','E',3)]
G.add_weighted_edges_from(l)
d=nx.minimum_spanning_tree(G)#调用求解最小生成树
for u,v,w in d.edges(data='weight'):
    print(f"{u}->{v},{w}")
#给出最小生成树每一条边
sum(w for u,v,w in d.edges(data='weight'))#求出最小生成树大小

A->C,1
B->C,2
B->D,5
D->E,3


11

In [ ]:
4. 拓扑排序（DAG）
题目名称：项目任务安排顺序
背景： 项目任务有前后依赖关系，边表示“先做哪个”，求可行的完成顺序。
调研 → 设计 → 编码 → 测试 → 发布
调研 → 采购硬件 → 安装设备


In [32]:
G = nx.DiGraph()
edges = [('调研', '设计'), ('设计', '编码'), ('编码', '测试'), ('测试', '发布'),
         ('调研', '采购硬件'), ('采购硬件', '安装设备')]
G.add_edges_from(edges)
order = list(nx.topological_sort(G))
print("项目完成顺序:", order)


项目完成顺序: ['调研', '设计', '采购硬件', '编码', '安装设备', '测试', '发布']


In [ ]:
5. 连通分量(并查集)
题目名称：网络中断检测
背景： 网络节点之间有连接，检测因中断导致的孤立子网络（连通块）。
组件1: 1 - 2 - 3  
组件2: 4 - 5  
组件3: 6


In [33]:
G = nx.Graph()
edges = [(1, 2), (2, 3), (4, 5)]
G.add_edges_from(edges)
G.add_node(6)  # 增加一个孤立点

components = list(nx.connected_components(G))
print("连通分量:")
for i, comp in enumerate(components, 1):
    print(f"第{i}个网络: {comp}")


连通分量:
第1个网络: {1, 2, 3}
第2个网络: {4, 5}
第3个网络: {6}


In [ ]:
6. 网络最大流
题目名称：河道最大运水量
背景： 给定水渠容量图，求从源点 S 到汇点 T 的最大流量。
S → A (10), S → B (5)
A → B (15), A → C (10)
B → D (10)
C → T (10)
D → T (10)


In [36]:
G = nx.DiGraph()
edges = [('S', 'A', 10), ('S', 'B', 5), ('A', 'B', 15),
         ('A', 'C', 10), ('B', 'D', 10), ('C', 'T', 10), ('D', 'T', 10)]
G.add_edges_from((u, v, {'capacity': c}) for u, v, c in edges)#给每一条边设置一个流量上限c
flow_value,flow_dict=nx.maximum_flow(G, 'S', 'T')
print("最大水流量:", flow_value)


最大水流量: 15


In [ ]:
7. 二分图匹配
题目名称：工作岗位分配
背景： 有员工和岗位的适配关系，求最多人匹配上合适岗位。
员工: A, B, C
岗位: 1, 2, 3

匹配边:
A - 1  
A - 2  
B - 1  
C - 3


In [41]:
import networkx as nx

# 创建图
G = nx.Graph()
edges = [('A', '1'), ('A', '2'), ('B', '1'), ('C', '3')]
G.add_edges_from(edges)

# 确保图是二分图
# 使用 bipartite 库将图划分为两个集合
left_nodes = {'A', 'B', 'C'}  # 左集合
right_nodes = {'1', '2', '3'}  # 右集合

# 计算最大匹配
matching = nx.bipartite.maximum_matching(G, top_nodes=left_nodes)

# 打印匹配结果
print("最大匹配:")
for u, v in matching.items():
    if u < v:  # 避免重复打印
        print(f"{u} → {v}")


最大匹配:
3 → C
1 → B
2 → A


In [ ]:
8. 图的着色（染色问题）
题目名称：会议安排冲突检测
背景： 有冲突的会议不能同一时间开，求最少使用多少个会议室。
会议冲突图（边表示冲突）：
1 - 2, 1 - 3, 2 - 4, 3 - 4


In [38]:
G = nx.Graph()
edges = [(1, 2), (1, 3), (2, 4), (3, 4)]
G.add_edges_from(edges)

coloring = nx.coloring.greedy_color(G, strategy='largest_first')
print("会议室安排（颜色表示会议室）:")
print(coloring)


会议室安排（颜色表示会议室）:
{1: 0, 2: 1, 3: 1, 4: 0}


In [ ]:
9. Euler 回路 / Hamilton 回路
题目名称：回收路线设计
背景： 清洁车要遍历所有道路（Euler 回路）或所有城市一次（Hamilton 回路）。
A - B - C - D - A（四边形）


In [39]:
G = nx.Graph()
edges = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'A')]
G.add_edges_from(edges)

if nx.is_eulerian(G):
    cycle = list(nx.eulerian_circuit(G))
    print("Euler 回路路径:")
    print(cycle)
else:
    print("图不是 Euler 图")


Euler 回路路径:
[('A', 'D'), ('D', 'C'), ('C', 'B'), ('B', 'A')]


In [ ]:
10. 强连通分量（SCC）
题目名称：网站相互可达性分析
背景： 网站间互相跳转，求哪些网站之间可以互达（强连通子图）。
A → B → C → A（互相可达）
C → D（不可返回）
D → E → D（互相可达）


In [40]:
G = nx.DiGraph()
edges = [('A', 'B'), ('B', 'C'), ('C', 'A'),
         ('C', 'D'), ('D', 'E'), ('E', 'D')]
G.add_edges_from(edges)

scc = list(nx.strongly_connected_components(G))
print("强连通分量:")
for i, comp in enumerate(scc, 1):
    print(f"第{i}组: {comp}")


强连通分量:
第1组: {'D', 'E'}
第2组: {'B', 'C', 'A'}


In [ ]:
11.最小费用流
假设你是一个物流公司负责人，你需要设计一条运输路线，使得从仓库（源点）到分配中心（汇点）运输一定数量的货物，并且最小化运输的总费用。以下是一个运输网络：

仓库（源点：S）到城市A的容量为 10，费用为 4。

仓库（源点：S）到城市B的容量为 8，费用为 6。

城市A到城市B的容量为 5，费用为 2。

城市A到城市C的容量为 7，费用为 3。

城市B到城市C的容量为 10，费用为 1。

城市C到分配中心（汇点：T）的容量为 10，费用为 5。

你的目标是设计最优的运输路线，运输的总流量为 10 单位流量，从仓库 S 到分配中心 T，并且使运输的总费用最小。

节点：S（源点）、A、B、C、T（汇点）

边：(S, A, 10, 4), (S, B, 8, 6), (A, B, 5, 2), (A, C, 7, 3), (B, C, 10, 1), (C, T, 10, 5)

In [80]:
import networkx as nx

# 创建有向图
G = nx.DiGraph()

# 添加带有容量和费用的边 (起点, 终点, 容量, 费用)
edges = [
    ('S', 'A', 10, 4),  # (起点, 终点, 容量, 费用)
    ('S', 'B', 8, 6),
    ('A', 'B', 5, 2),
    ('A', 'C', 7, 3),
    ('B', 'C', 10, 1),
    ('C', 'T', 10, 5)
]

# 添加边到图中
for u, v, capacity, cost in edges:
    G.add_edge(u, v, capacity=capacity, weight=cost)

# 设置流量需求 (源点S流出10，汇点T流入10，其他节点流量需求为0)
demand = {'S': -10, 'T': 10, 'A': 0, 'B': 0, 'C': 0}

# 将流量需求添加到节点属性中,给每一个点都设置
for node, dem in demand.items():
    G.nodes[node]['demand'] = dem

# 使用 min_cost_flow 计算最小费用流
flow_dict_min_cost = nx.min_cost_flow(G)

# 计算最小费用流的总费用
total_cost = nx.cost_of_flow(G, flow_dict_min_cost)

# 输出流量分配和最小费用
print("流量分配:", flow_dict_min_cost)
print("最小费用:", total_cost)





流量分配: {'S': {'A': 10, 'B': 0}, 'A': {'B': 5, 'C': 5}, 'B': {'C': 5}, 'C': {'T': 10}, 'T': {}}
最小费用: 120
